In [ ]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import sklearn

# things i'll use for SVD
from collections import defaultdict
from numpy import linalg

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

import tensorflow as tf

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
gist_url = "https://gist.githubusercontent.com/GottaGitGoing/c01194dc318f3a31a07d1ae68d31b163/raw/ec37b65f0aa79cf89480981cd9be0df6399dfafb/customer_orders.csv"
cust_order = pd.read_csv(gist_url,low_memory=False)
MAX_CUST = 200

In [ ]:
df = pd.DataFrame(data=cust_order)

In [ ]:
df

,customer_id,order_id,weather,order_timestamp,item_id,item_name,taste_profile,item_type
0,52,1752,snowy,2015-01-02 20:38:35,640405323,Small Curly Fries,savory,side
1,52,1752,snowy,2015-01-02 20:38:35,640405331,Regular Curly Fries,savory,side
2,194,244,snowy,2015-01-04 16:51:57,640405315,Large Fries,savory,side
3,1,714,cloudy,2015-01-05 20:58:57,640405307,Regular Fries,savory,side
4,1,714,cloudy,2015-01-05 20:58:57,640405355,Small Drink,chilling,beverage
...,...,...,...,...,...,...,...,...
4804,178,1088,rainy,2020-12-26 08:02:53,640405395,Regular Shake,sweet,beverage
4805,122,1290,sunny,2020-12-28 04:46:26,640404923,Hamburger,savory,burger
4806,122,1290,sunny,2020-12-28 04:46:26,640404963,Cheeseburger,savory,burger
4807,122,1290,sunny,2020-12-28 04:46:26,640405085,Double Hamburger,savory,burger


In [ ]:
df[['customer_id','item_id']]

,customer_id,item_id
0,52,640405323
1,52,640405331
2,194,640405315
3,1,640405307
4,1,640405355
...,...,...
4804,178,640405395
4805,122,640404923
4806,122,640404963
4807,122,640405085


In [ ]:
order_dictionary = defaultdict(dict)
for c_id, i_id in zip(df['customer_id'],df['item_id']):
  if i_id not in order_dictionary[c_id]:
    order_dictionary[c_id][i_id] = 1
  else:
    order_dictionary[c_id][i_id]+=1

In [ ]:
item_ids = [
        640404923,
        640404963,
        640405025,
        640405058,
        640405085,
        640405112,
        640405172,
        640405355,
        640405371,
        640405380,
        640405389,
        640405395,
        640405399,
        640405296,
        640405307,
        640405315,
        640405323,
        640405331,
        640405339,
        640405347,
        640405348
    ]


In [ ]:
# print(sorted(order_dictionary[52]))
# print(sorted(order_dictionary[122]))
# sorted users, sorted items

# print(sorted(order_dictionary.keys()))
# for i in sorted(order_dictionary):
#   print(sorted(order_dictionary[i]))
dim = (MAX_CUST+1,len(item_ids)) # +1 because lazy attempt at not wanting to deal with 0 being start not 1 in computers
sorted_item_indx = sorted(item_ids) 
index_dictionary = { j:i for i,j in enumerate(sorted(item_ids))}
the_matrix = np.zeros(dim)

In [ ]:
print(order_dictionary)

defaultdict(<class 'dict'>, {52: {640405323: 3, 640405331: 1, 640405085: 3, 640405172: 2, 640405347: 5, 640405339: 1, 640405112: 3, 640405058: 1, 640405399: 2, 640405380: 2, 640404923: 1, 640405307: 1, 640404963: 3, 640405395: 1, 640405389: 1, 640405025: 2}, 194: {640405315: 1, 640405380: 2, 640405331: 4, 640405355: 1, 640404963: 1, 640405348: 2, 640405058: 1, 640405172: 1, 640405112: 1, 640405296: 4, 640405399: 1, 640405323: 1, 640405025: 1, 640405395: 2, 640405347: 1}, 1: {640405307: 2, 640405355: 3, 640404963: 2, 640405347: 4, 640404923: 3, 640405339: 3, 640405058: 3, 640405296: 2, 640405399: 4, 640405112: 1, 640405172: 1, 640405389: 1, 640405380: 1, 640405085: 1}, 47: {640405371: 2, 640405395: 1, 640405085: 1, 640405058: 1, 640404923: 1, 640405025: 1, 640405348: 1, 640405331: 1, 640405172: 1, 640405399: 1, 640405315: 1, 640405323: 1, 640405389: 1}, 192: {640404963: 1, 640405315: 1, 640405348: 2, 640405085: 2, 640405339: 3, 640405112: 1, 640405172: 2, 640405347: 1, 640405380: 1, 640

In [ ]:
print(index_dictionary)

{640404923: 0, 640404963: 1, 640405025: 2, 640405058: 3, 640405085: 4, 640405112: 5, 640405172: 6, 640405296: 7, 640405307: 8, 640405315: 9, 640405323: 10, 640405331: 11, 640405339: 12, 640405347: 13, 640405348: 14, 640405355: 15, 640405371: 16, 640405380: 17, 640405389: 18, 640405395: 19, 640405399: 20}


In [ ]:
# fill in the user-item matrix
for cust,orders in order_dictionary.items():
  # print(cust,orders)
  # the_matrix[row being user][column being item rating]
  for itm_id, rating in orders.items():
    the_matrix[cust][index_dictionary[itm_id]] = rating

In [ ]:
the_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [3., 2., 0., ..., 1., 0., 4.],
       [2., 0., 0., ..., 1., 0., 2.],
       ...,
       [1., 0., 2., ..., 2., 4., 3.],
       [1., 1., 0., ..., 2., 2., 2.],
       [1., 2., 1., ..., 0., 2., 0.]])

In [ ]:
M = pd.DataFrame(the_matrix,columns=item_ids,index=[i for i in range(201)]) # 201 because human index starts at 1 not 0

In [ ]:
M

,640404923,640404963,640405025,640405058,640405085,640405112,640405172,640405355,640405371,640405380,...,640405395,640405399,640405296,640405307,640405315,640405323,640405331,640405339,640405347,640405348
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,2.0,0.0,3.0,1.0,1.0,1.0,2.0,2.0,0.0,...,0.0,3.0,4.0,0.0,3.0,0.0,1.0,1.0,0.0,4.0
2,2.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,1.0,...,2.0,3.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0
3,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,0.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.0,1.0,1.0,3.0,0.0,2.0,1.0,1.0,0.0,2.0,...,5.0,2.0,1.0,4.0,3.0,1.0,2.0,0.0,2.0,0.0
197,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0
198,1.0,0.0,2.0,1.0,2.0,4.0,0.0,2.0,3.0,2.0,...,2.0,2.0,1.0,0.0,0.0,1.0,3.0,2.0,4.0,3.0
199,1.0,1.0,0.0,4.0,1.0,2.0,1.0,1.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,2.0,2.0


In [ ]:
# Attempting to normalize/ fixing having all those zeros in the matrix
# trying the easier imputation (baseline predictor)
r_average = M[M>0.0].mean()
M[M==0] = np.NaN
M.fillna(r_average, inplace=True)

In [ ]:
M

,640404923,640404963,640405025,640405058,640405085,640405112,640405172,640405355,640405371,640405380,...,640405395,640405399,640405296,640405307,640405315,640405323,640405331,640405339,640405347,640405348
0,1.640845,1.807407,1.637681,1.734848,1.666667,1.691781,1.740741,1.539568,1.757576,1.612403,...,1.555556,1.837037,1.644444,1.712329,1.78626,1.753623,1.706349,1.72093,1.625,1.716418
1,3.000000,2.000000,1.637681,3.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.612403,...,1.555556,3.000000,4.000000,1.712329,3.00000,1.753623,1.000000,1.00000,1.625,4.000000
2,2.000000,1.807407,1.637681,1.734848,1.000000,2.000000,2.000000,1.539568,1.757576,1.000000,...,2.000000,3.000000,1.000000,1.712329,2.00000,1.753623,1.706349,1.00000,1.625,2.000000
3,1.000000,1.807407,1.637681,2.000000,1.000000,1.691781,1.740741,1.539568,1.757576,1.000000,...,1.000000,1.000000,1.000000,1.712329,1.78626,1.753623,1.706349,1.00000,1.000,1.000000
4,1.640845,1.000000,2.000000,1.734848,1.666667,1.691781,1.000000,1.000000,2.000000,1.612403,...,1.000000,1.837037,1.644444,1.712329,1.78626,1.753623,1.706349,2.00000,1.625,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,1.640845,1.000000,1.000000,3.000000,1.666667,2.000000,1.000000,1.000000,1.757576,2.000000,...,5.000000,2.000000,1.000000,4.000000,3.00000,1.000000,2.000000,1.72093,2.000,1.716418
197,1.640845,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.539568,1.757576,1.000000,...,1.000000,1.000000,1.000000,1.000000,2.00000,1.000000,1.706349,1.72093,1.625,2.000000
198,1.000000,1.807407,2.000000,1.000000,2.000000,4.000000,1.740741,2.000000,3.000000,2.000000,...,2.000000,2.000000,1.000000,1.712329,1.78626,1.000000,3.000000,2.00000,4.000,3.000000
199,1.000000,1.000000,1.637681,4.000000,1.000000,2.000000,1.000000,1.000000,1.757576,1.000000,...,1.000000,1.000000,1.644444,1.712329,1.00000,1.000000,1.706349,2.00000,2.000,2.000000


In [ ]:
U, sigma, Vt = linalg.svd(M)


In [ ]:
# for the sake of reducing the matrix
def rank_k(k):
  U_reduced = np.mat(U[:,:k])
  Vt_reduced = np.mat(Vt[:k,:])
  sigma_reduced = np.eye(k)*sigma[:k]
  return U_reduced, sigma_reduced, Vt_reduced

In [ ]:
U_reduced, sigma_reduced, Vt_reduced = rank_k(4)
M_hat = U_reduced * sigma_reduced * Vt_reduced

In [ ]:
M_hat

matrix([[1.63085595, 1.80919908, 1.6333647 , ..., 1.71728071, 1.61819044,
         1.72292047],
        [1.43242445, 2.05855969, 2.31013674, ..., 2.15655011, 1.37984249,
         2.29752385],
        [1.61490184, 1.72908813, 1.54385992, ..., 1.80603212, 1.59795228,
         1.54829298],
        ...,
        [2.44955532, 2.70109714, 2.12005288, ..., 1.7013181 , 2.4734767 ,
         2.75647313],
        [1.30902241, 1.92265605, 1.61361607, ..., 1.58520043, 1.26845707,
         2.10946159],
        [1.6865852 , 1.89359758, 1.27934607, ..., 1.64034718, 1.66987195,
         1.70662099]])

In [ ]:
M_hat_matrix = pd.DataFrame(M_hat,columns=item_ids,index=[i for i in range(201)])

In [ ]:
M_hat_matrix

,640404923,640404963,640405025,640405058,640405085,640405112,640405172,640405355,640405371,640405380,...,640405395,640405399,640405296,640405307,640405315,640405323,640405331,640405339,640405347,640405348
0,1.630856,1.809199,1.633365,1.735445,1.660488,1.699475,1.745905,1.529320,1.758454,1.612714,...,1.550964,1.833089,1.639408,1.715495,1.785793,1.769310,1.715585,1.717281,1.618190,1.722920
1,1.432424,2.058560,2.310137,2.742138,1.829444,1.931466,0.752519,1.535151,2.043876,1.621358,...,2.069886,2.626551,2.772457,2.073680,2.389543,2.165308,2.158145,2.156550,1.379842,2.297524
2,1.614902,1.729088,1.543860,1.725262,1.633292,1.465053,1.942110,1.594300,1.737632,1.569036,...,1.523418,1.991104,1.559124,1.609714,1.894172,1.929285,1.751199,1.806032,1.597952,1.548293
3,1.409029,1.586328,1.366794,1.482694,1.391807,1.429813,1.455950,1.306603,1.522215,1.350947,...,1.336477,1.503297,1.353226,1.475859,1.438758,1.466687,1.478999,1.461360,1.396983,1.513486
4,1.286741,1.422328,1.643789,1.657583,1.559232,1.612608,1.429441,1.292125,1.465156,1.492301,...,1.367402,1.944429,1.808741,1.494289,1.979014,1.743000,1.476948,1.551084,1.274577,1.416824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,1.708826,2.245208,2.040172,2.389095,1.787322,1.961392,1.155712,1.618619,2.113251,1.659355,...,1.984798,2.096722,2.214278,2.105292,1.883308,1.883690,2.103104,2.015324,1.671974,2.362242
197,1.409036,1.603155,1.220338,1.401844,1.283099,1.278103,1.424525,1.286915,1.508533,1.245427,...,1.298831,1.334007,1.151178,1.419260,1.214136,1.352920,1.459694,1.413957,1.395154,1.498998
198,2.449555,2.701097,2.120053,1.768639,2.235688,3.093202,2.186449,1.828284,2.337945,2.340277,...,1.917743,1.267138,1.768949,2.561393,1.489820,1.454748,1.939912,1.701318,2.473477,2.756473
199,1.309022,1.922656,1.613616,2.040762,1.299234,1.507934,0.474221,1.212359,1.742088,1.169169,...,1.668728,1.504043,1.779155,1.735962,1.221697,1.316924,1.733865,1.585200,1.268457,2.109462


In [ ]:
def print_top_k_recommendations(user_id: int, k:int):
  # basically given a user from M_hat_matrix, print the top k item_ids
  assert user_id <= MAX_CUST
  the_mat = M_hat_matrix.iloc[[user_id]]
  assert k<len(item_ids)
  the_dic = dict()
  tposed = the_mat.T
  for i in tposed.index:
    itm_id = i
    rtng = tposed.loc[itm_id,user_id]
    the_dic[itm_id] = rtng
  # return {k: v for k, v in sorted(the_dic.items(), key=lambda item: item[1],reverse=True)}
  return sorted(the_dic.items(),key=lambda x: -x[1])[:k]
  

  # make a dictionary of item_id, rating
  # pred_dict = {itm_id:rtng for itm_id, rtng in M}


In [ ]:
# user_id = 1
# the_mat = M_hat_matrix.iloc[[user_id]]
# tposed = the_mat.T
# for i in tposed.index:
  # itm_id = i
  # rtng = tposed.loc[itm_id,user_id]

# for i_id,rtng in zip(tposed['customer_id'],tposed['item_id']):
# 
print_top_k_recommendations(21,5)

[(640405112, 2.678275330801895),
 (640405296, 2.2670566326497474),
 (640405315, 2.2492668561887212),
 (640405025, 2.1333797077707035),
 (640405380, 2.005985506940481)]

In [ ]:
# df[["item_id","item_name"]]
item_name_dict = dict()

for it_name, i_id in zip(df['item_name'],df['item_id']):
  item_name_dict[i_id] = it_name

In [ ]:
cc = print_top_k_recommendations(52,20)
for i,j in cc:
  print(i,j,item_name_dict[i])

640405112 3.463544854345002 Double Cheeseburger
640405296 2.771248856656141 Small Fries
640405025 2.5833103433364353 Bacon Cheeseburger
640405315 2.343669978606351 Large Fries
640405380 2.2892393325602307 Large Drink
640405389 2.2435604537668263 Small Shake
640405307 2.243433868110351 Regular Fries
640405085 2.2297949270043134 Double Hamburger
640405348 2.2276163165262655 Coffee
640405058 1.7929707580022984 Veggie Burger
640404963 1.767624580648162 Cheeseburger
640405399 1.747451117247628 Large Shake
640405371 1.6400404369755424 Regular Drink
640405347 1.5916876418230799 Onion Rings
640405395 1.5747168479926203 Regular Shake
640404923 1.5597896704623038 Hamburger
640405323 1.4413507342956897 Small Curly Fries
640405331 1.3363853548222613 Regular Curly Fries
640405339 1.2786802020378727 Large Curly Fries
640405355 1.170373240293146 Small Drink


## Attempting A different imputation technique
## Using Baseline Predictors over averaging out the zeros
# IDEA SCRAPPED FOR NOW!
____

In [ ]:
global_mean = M[M>0].mean().mean()
M_minus_mean = M[M>0]-global_mean
user_bias = M_minus_mean.T.mean()
item_bias = M_minus_mean.apply(lambda x: x - user_bias).mean()

In [ ]:
# for i in user_bias:
#   print(i)
user_bias


0     -4.229421e-16
1      3.337680e-01
2     -1.647169e-02
3     -2.612844e-01
4     -1.527953e-01
           ...     
196    2.801023e-01
197   -3.284378e-01
198    4.488785e-01
199   -1.632897e-01
200   -1.433637e-01
Length: 201, dtype: float64

In [ ]:
item_bias

640404923   -0.055359
640404963    0.111203
640405025   -0.058523
640405058    0.038644
640405085   -0.029538
640405112   -0.004423
640405172    0.044537
640405355   -0.156636
640405371    0.061372
640405380   -0.083801
640405389    0.036620
640405395   -0.140649
640405399    0.140833
640405296   -0.051760
640405307    0.016125
640405315    0.090055
640405323    0.057419
640405331    0.010145
640405339    0.024726
640405347   -0.071204
640405348    0.020214
dtype: float64

# To Implement: Create Hybrid recommender (Ryan's and Mine combined)

# IMPLEMTING THE BILL Schtein 